In [13]:
from fenics import plot, Point, CompiledSubDomain, MeshFunction, XDMFFile
from mshr import Rectangle, generate_mesh
from fenics import SubDomain

inlet_id = 1
top_id = 2
outlet_id = 3
bottom_id = 4

id_fluid = 5
id_pipe_walls = 6


def create_mesh(length, height_fluid, pipe_thickness, refinement=1000):
    p1 = Point(0, 0)
    p2 = Point(height_fluid, length)
    fluid_rectangle = Rectangle(p1, p2)

    p1 = Point(height_fluid, 0)
    p2 = Point(height_fluid + pipe_thickness, length)
    pipe_rectangle = Rectangle(p1, p2)

    domain = fluid_rectangle + pipe_rectangle

    domain.set_subdomain(1, fluid_rectangle)
    domain.set_subdomain(2, pipe_rectangle)
    mesh = generate_mesh(domain, refinement)

    # marking physical groups (volumes and surfaces)
    volume_markers = MeshFunction("size_t", mesh, mesh.topology().dim())
    volume_markers.set_all(1)

    tol = 1e-14

    id_fluid = 5
    id_pipe_walls = 6

    class Fluid(SubDomain):
        def inside(self, x, on_boundary):
            return x[0] <= height_fluid + tol

    class Pipe(SubDomain):
        def inside(self, x, on_boundary):
            return x[0] >= height_fluid - tol

    fluid = Fluid()
    pipe = Pipe()
    # marking volumes
    fluid.mark(volume_markers, id_fluid)
    pipe.mark(volume_markers, id_pipe_walls)

    tol = 1e-14

    inlet_surface = CompiledSubDomain(
        "on_boundary && near(x[1], 0, tol) && x[0] < height_fluid + tol",
        tol=tol,
        height_fluid=height_fluid,
    )
    outlet_surface = CompiledSubDomain(
        "on_boundary && near(x[1], outlet_x, tol) && x[0] < height_fluid + tol",
        tol=tol,
        height_fluid=height_fluid,
        outlet_x=length,
    )
    bottom_surface = CompiledSubDomain("on_boundary && near(x[0], 0, tol)", tol=tol)
    top_surface = CompiledSubDomain(
        "on_boundary && near(x[0], top_y, tol)",
        tol=tol,
        top_y=height_fluid + pipe_thickness,
    )

    surface_markers = MeshFunction("size_t", mesh, mesh.topology().dim() - 1)
    surface_markers.set_all(0)

    inlet_surface.mark(surface_markers, inlet_id)
    outlet_surface.mark(surface_markers, outlet_id)
    top_surface.mark(surface_markers, top_id)
    bottom_surface.mark(surface_markers, bottom_id)

    output_file = XDMFFile("challenge_E/surface_markers.xdmf")
    output_file.write(surface_markers)

    output_file2 = XDMFFile("challenge_E/volume_markers.xdmf")
    output_file2.write(volume_markers)

In [14]:
import fenics as f
import festim as F
import h_transport_materials as htm

# PbLi properties
D_pbli = (
    htm.diffusivities.filter(material="lipb")
    .filter(isotope="h")
    .filter(author="reiter")
)[0]
S_pbli = (
    htm.solubilities.filter(material="lipb").filter(isotope="h").filter(author="aiello")
)[0]

# eurofer properties
D_eurofer = htm.diffusivities.filter(material="eurofer_97").filter(author="chen")[0]
S_eurofer = htm.solubilities.filter(material="eurofer_97").filter(author="chen")[0]


def run_model(temperature, length, height_fluid, pipe_thickness, velocity):
    my_model = F.Simulation()

    # create mesh
    create_mesh(length, height_fluid, pipe_thickness)

    my_model.mesh = F.MeshFromXDMF(
        volume_file="challenge_E/volume_markers.xdmf",
        boundary_file="challenge_E/surface_markers.xdmf",
        type="cylindrical",
    )
    # materials
    eurofer = F.Material(
        id=id_pipe_walls,
        D_0=D_eurofer.pre_exp.magnitude,
        E_D=D_eurofer.act_energy.magnitude,
        S_0=S_eurofer.pre_exp.magnitude,
        E_S=S_eurofer.act_energy.magnitude,
    )
    lipb = F.Material(
        id=id_fluid,
        D_0=D_pbli.pre_exp.magnitude,
        E_D=D_pbli.act_energy.magnitude,
        S_0=S_pbli.pre_exp.magnitude,
        E_S=S_pbli.act_energy.magnitude,
    )
    my_model.materials = F.Materials([eurofer, lipb])

    my_model.T = F.Temperature(temperature)

    my_model.boundary_conditions = [
        F.DirichletBC(field="solute", surfaces=top_id, value=0),
        F.DirichletBC(field="solute", surfaces=inlet_id, value=1e18),
    ]

    c_in = F.AverageSurface("solute", inlet_id)
    c_out = F.AverageSurface("solute", outlet_id)

    derived_quantities = F.DerivedQuantities([c_in, c_out])

    my_model.exports = F.Exports(
        [F.XDMFExport("solute", folder="challenge_E/", mode=1), derived_quantities]
    )

    my_model.settings = F.Settings(
        absolute_tolerance=1e10,
        relative_tolerance=1e-10,
        transient=False,
        chemical_pot=True,
    )
    my_model.initialise()

    # adding advection term:
    mesh_sub = f.SubMesh(my_model.mesh.mesh, my_model.mesh.volume_markers, id_fluid)

    functionspace = f.VectorFunctionSpace(mesh_sub, "CG", 1)

    velocity = f.Expression(
        ("0", "v*(x[0] - height_fluid)*(x[0] + height_fluid)"),
        v=velocity,
        height_fluid=height_fluid,
        degree=2,
    )

    velocity = f.interpolate(velocity, functionspace)

    V = f.VectorFunctionSpace(my_model.mesh.mesh, "CG", 1)
    u = f.Function(V)
    v = f.TestFunction(V)

    form = f.inner(u, v) * my_model.mesh.dx
    form += f.inner(velocity, v) * my_model.mesh.dx(id_fluid)
    f.solve(form == 0, u, bcs=[])

    velocity = u

    XDMFFile("challenge_E/velocity_field.xdmf").write(u)
    (
        hydrogen_concentration,
        _,
    ) = my_model.h_transport_problem.mobile.get_concentration_for_a_given_material(
        lipb, my_model.T
    )
    test_function_mobile = my_model.h_transport_problem.mobile.test_function
    advection_term = f.inner(
        f.dot(f.grad(hydrogen_concentration), u), test_function_mobile
    ) * my_model.mesh.dx(id_fluid)
    my_model.h_transport_problem.F += advection_term

    my_model.run()

    return c_out, c_in


def compute_efficiency(temperature, length, height_fluid, pipe_thickness, velocity):
    c_out, c_in = run_model(temperature, length, height_fluid, pipe_thickness, velocity)
    c_out_value = c_out.data[0]
    c_in_value = c_in.data[0]
    efficiency = 1 - c_out_value / c_in_value
    return efficiency

In [15]:
print(
    compute_efficiency(
        temperature=800,
        length=0.3,
        height_fluid=1e-2,
        pipe_thickness=4e-3,
        velocity=40e-2,
    )
)

Succesfully load mesh with 12008 cells
Defining initial values
Defining variational problem
Defining source terms
Defining boundary conditions
Solving steady state problem...
Solved problem in 0.20 s
0.7794751825839962


/home/jdark/anaconda3/envs/festim-env/lib/python3.11/site-packages/festim/exports/derived_quantities/derived_quantities.py:129: DeprecationWarning: The current derived_quantities title style will be deprecated in a future release, please use show_units=True instead
  warnings.warn(
/home/jdark/anaconda3/envs/festim-env/lib/python3.11/site-packages/festim/exports/derived_quantities/derived_quantities.py:129: DeprecationWarning: The current derived_quantities title style will be deprecated in a future release, please use show_units=True instead
  warnings.warn(
